In [1]:
import pandas as pd

In [2]:
flights = pd.read_csv('flights.csv',low_memory=False)
airports = pd.read_csv('airports.csv')
airlines = pd.read_csv('airlines.csv')

all_features = flights.columns.values
all_features

array(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
       'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME',
       'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN',
       'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED',
       'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY',
       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
       'WEATHER_DELAY'], dtype=object)

In [3]:
# Восстанавливаем данные по аэропортам прибытия и отправления, т.к. часть данных указана кодом в 3х-значном формате и часть в формате ID
aircode1 = pd.read_csv('L_AIRPORT.csv')
aircode2 = pd.read_csv('L_AIRPORT_ID.csv')

aircode1 = aircode1.reset_index()
aircode2 = aircode2.reset_index()
aircodes = pd.merge(aircode1,aircode2,on='Description')
aircode_dict = dict(zip(aircodes['Code_y'].astype(str),aircodes['Code_x']))

In [4]:
# Необходимо, чтобы все аэропорты отправления и прибытия являлись строками
flights['ORIGIN_AIRPORT'] = flights['ORIGIN_AIRPORT'].values.astype(str)
flights['DESTINATION_AIRPORT'] = flights['DESTINATION_AIRPORT'].values.astype(str)

for i in range(len(flights)):
    if len(flights['ORIGIN_AIRPORT'][i]) != 3:
        to_replace = flights['ORIGIN_AIRPORT'][i]
        value = aircode_dict[flights['ORIGIN_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)
for i in range(len(flights)):
    if len(flights['DESTINATION_AIRPORT'][i]) != 3:
        to_replace = flights['DESTINATION_AIRPORT'][i]
        value = aircode_dict[flights['DESTINATION_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)

In [5]:
len(flights['ORIGIN_AIRPORT'][i]) != 3

False

In [6]:
len(flights['DESTINATION_AIRPORT'][i]) != 3

False

In [7]:
flights.shape

(5819079, 31)

In [8]:
df = flights.merge(airlines, left_on = 'AIRLINE',right_on = 'IATA_CODE')
df.shape

(5819079, 33)

In [9]:
df_all = df.merge(airports, left_on = ['ORIGIN_AIRPORT'], right_on = ['IATA_CODE'],how = 'outer')

df_all.shape

(5819079, 40)

In [10]:
df_all['DATE'] = pd.to_datetime(df[['YEAR','MONTH','DAY']],dayfirst=True)
df_all.drop(['YEAR','MONTH','DAY'],axis=1,inplace = True)

In [13]:
#!pip install sweetviz
import sweetviz as sv

my_report = sv.analyze(df_all)
my_report.show_html() 

                                             |      | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [11]:
df_all.drop_duplicates(inplace=True)

In [12]:
df_group = df_all.groupby(['ORIGIN_AIRPORT','AIRLINE_y'])['DEPARTURE_DELAY'].mean().reset_index().sort_values(by = 'DEPARTURE_DELAY')
df_group

,ORIGIN_AIRPORT,AIRLINE_y,DEPARTURE_DELAY
1308,XNA,American Airlines Inc.,-7.055556
4,ABQ,Alaska Airlines Inc.,-6.709589
857,MTJ,Atlantic Southeast Airlines,-6.086093
1313,YAK,Alaska Airlines Inc.,-6.070932
276,CNY,Skywest Airlines Inc.,-6.058537
...,...,...,...
174,BTR,United Air Lines Inc.,56.000000
1276,TUL,Virgin America,113.000000
199,BWI,Skywest Airlines Inc.,156.000000
411,EWR,Skywest Airlines Inc.,299.500000


In [13]:
df_all_merge = df_all.merge(df_group, how='left', on = ['ORIGIN_AIRPORT', 'AIRLINE_y'])

In [14]:
df_all_merge[df_all_merge.DEPARTURE_DELAY_x.isna()]

,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY_x,TAXI_OUT,...,AIRLINE_y,IATA_CODE_y,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,DATE,DEPARTURE_DELAY_y
3,4,AS,136,N431AS,ANC,SEA,135,NaN,NaN,NaN,...,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,2015-01-01,0.455896
233,3,AS,45,N763AS,ANC,BET,1905,NaN,NaN,NaN,...,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,2015-01-01,0.455896
349,7,AS,144,N535AS,ANC,PDX,200,NaN,NaN,NaN,...,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,2015-01-01,0.455896
400,1,AS,49,N765AS,ANC,ADQ,1530,NaN,NaN,NaN,...,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,2015-01-01,0.455896
811,7,AS,106,N767AS,ANC,SEA,2140,NaN,NaN,NaN,...,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,2015-01-02,0.455896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5816785,7,WN,3209,NaN,ISP,BWI,615,NaN,NaN,NaN,...,Southwest Airlines Co.,ISP,Long Island MacArthur Airport,Islip,NY,USA,40.79524,-73.10021,2015-12-22,10.078287
5818304,5,WN,1500,N792SW,ISP,BWI,935,NaN,NaN,NaN,...,Southwest Airlines Co.,ISP,Long Island MacArthur Airport,Islip,NY,USA,40.79524,-73.10021,2015-12-30,10.078287
5818317,6,WN,1434,N7752B,ISP,BWI,1215,NaN,NaN,NaN,...,Southwest Airlines Co.,ISP,Long Island MacArthur Airport,Islip,NY,USA,40.79524,-73.10021,2015-12-30,10.078287
5818418,7,WN,605,NaN,ISP,BWI,505,NaN,NaN,NaN,...,Southwest Airlines Co.,ISP,Long Island MacArthur Airport,Islip,NY,USA,40.79524,-73.10021,2015-12-31,10.078287


In [15]:
df_all_merge.loc[df_all_merge.DEPARTURE_DELAY_x.isna(),'DEPARTURE_DELAY_x'] = df_all_merge.DEPARTURE_DELAY_y

In [16]:
df_all_merge[df_all_merge.DEPARTURE_DELAY_x.isna()]
#df_all_merge.DEPARTURE_DELAY_x.isna().sum()

,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY_x,TAXI_OUT,...,AIRLINE_y,IATA_CODE_y,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,DATE,DEPARTURE_DELAY_y


In [17]:
del df_all_merge['DEPARTURE_DELAY_y']

In [18]:
df_all_merge.shape

(5818485, 38)

In [19]:
df_grouped = df_all_merge.groupby(['ORIGIN_AIRPORT','AIRPORT'])['DEPARTURE_DELAY_x'].mean().reset_index().sort_values(by = 'DEPARTURE_DELAY_x')
df_grouped.query('DEPARTURE_DELAY_x>0').head()

,ORIGIN_AIRPORT,AIRPORT,DEPARTURE_DELAY_x
46,BTM,Bert Mooney Airport,0.131171
10,ADQ,Kodiak Airport,0.268817
43,BRD,Brainerd Lakes Regional Airport,0.348031
89,DLG,Dillingham Airport,0.558442
184,LIH,Lihue Airport,0.598528


## Аэропорт с минимальной задержкой вылета (в среднем) - Bert Mooney Airport

In [20]:
airports = airports.query('CITY == "Los Angeles"')
airports

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
176,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807


In [21]:
df_all_merge.ARRIVAL_DELAY.isna().sum()

104494

In [22]:
df_group_arrival = df_all_merge.groupby(['DESTINATION_AIRPORT','AIRLINE_y'])['ARRIVAL_DELAY'].mean().reset_index().sort_values(by = 'ARRIVAL_DELAY')
df_group_arrival

,DESTINATION_AIRPORT,AIRLINE_y,ARRIVAL_DELAY
503,GSO,US Airways Inc.,-23.000000
192,BUR,United Air Lines Inc.,-22.000000
227,CHS,Alaska Airlines Inc.,-18.692308
620,JFK,Alaska Airlines Inc.,-18.542056
4,ABQ,Alaska Airlines Inc.,-15.789041
...,...,...,...
302,DAB,Atlantic Southeast Airlines,41.000000
710,LIT,American Airlines Inc.,42.875000
304,DAB,US Airways Inc.,44.000000
119,BMI,Frontier Airlines Inc.,44.444444


In [23]:
df_all_merge_arrival = df_all_merge.merge(df_group_arrival, how='left', on = ['DESTINATION_AIRPORT', 'AIRLINE_y'])

In [24]:
df_all_merge_arrival.loc[df_all_merge_arrival.ARRIVAL_DELAY_x.isna(),'ARRIVAL_DELAY_x'] = df_all_merge_arrival.ARRIVAL_DELAY_y

In [25]:
df_all_merge_arrival[df_all_merge_arrival.ARRIVAL_DELAY_x.isna()]

,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY_x,TAXI_OUT,...,AIRLINE_y,IATA_CODE_y,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,DATE,ARRIVAL_DELAY_y


In [26]:
del df_all_merge_arrival['ARRIVAL_DELAY_y']

In [27]:
df_grouped_arrival = df_all_merge_arrival.query('DESTINATION_AIRPORT == "LAX"').groupby(['AIRLINE_y'])['ARRIVAL_DELAY_x'].mean().reset_index().sort_values(by = 'ARRIVAL_DELAY_x')
df_grouped_arrival.head()

,AIRLINE_y,ARRIVAL_DELAY_x
5,Hawaiian Airlines Inc.,-4.653317
0,Alaska Airlines Inc.,-0.525700
6,JetBlue Airways,-0.022152
3,Delta Air Lines Inc.,1.398882
1,American Airlines Inc.,2.122856


## Cамая пунктуальная авиакомпания по прилету в аэропорт Los Angeles - Hawaiian Airlines Inc. (прилетает раньше запланированного - с позиции пассажира это лучше, быстрее). Если рассматривать как с позиции пассажира, так и с позиции бизнеса- это Jet Blue Airways

In [28]:
df_all_merge_arrival.isna().sum()

DAY_OF_WEEK                  0
AIRLINE_x                    0
FLIGHT_NUMBER                0
TAIL_NUMBER              14234
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME           85576
DEPARTURE_DELAY_x            0
TAXI_OUT                 88470
WHEELS_OFF               88470
SCHEDULED_TIME               6
ELAPSED_TIME            104494
AIR_TIME                104494
DISTANCE                     0
WHEELS_ON                91936
TAXI_IN                  91936
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME             91936
ARRIVAL_DELAY_x              0
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5729178
AIR_SYSTEM_DELAY       4755046
SECURITY_DELAY         4755046
AIRLINE_DELAY          4755046
LATE_AIRCRAFT_DELAY    4755046
WEATHER_DELAY          4755046
IATA_CODE_x                  0
AIRLINE_y                    0
IATA_CODE_y               4021
AIRPORT                   4021
CITY    

In [29]:
df_taxi_grouped = df_all_merge_arrival.groupby(['ORIGIN_AIRPORT','AIRLINE_y'])['TAXI_OUT'].mean().reset_index().sort_values(by = 'TAXI_OUT')

In [30]:
df_taxi_grouped

,ORIGIN_AIRPORT,AIRLINE_y,TAXI_OUT
28,AGS,United Air Lines Inc.,5.000000
276,CNY,Skywest Airlines Inc.,6.512195
22,ADQ,Alaska Airlines Inc.,6.591398
158,BRW,Alaska Airlines Inc.,6.901982
707,LIH,Hawaiian Airlines Inc.,7.021047
...,...,...,...
693,LGA,Delta Air Lines Inc.,29.071429
623,JFK,Delta Air Lines Inc.,29.455759
692,LGA,Atlantic Southeast Airlines,31.492079
455,FSD,United Air Lines Inc.,33.000000


In [31]:
df_taxi = df_all_merge_arrival.merge(df_taxi_grouped, how='left', on = ['ORIGIN_AIRPORT', 'AIRLINE_y'])

In [32]:
df_taxi.loc[df_taxi.TAXI_OUT_x.isna(),'TAXI_OUT_x'] = df_taxi.TAXI_OUT_y

In [33]:
#df_taxi.TAXI_OUT_x.isna().sum()

In [34]:
del df_taxi['TAXI_OUT_y']

In [35]:
airport_counts = df_taxi.ORIGIN_AIRPORT.value_counts()
airport_counts 

ATL    379418
ORD    313500
DFW    260588
DEN    214186
LAX    212395
        ...  
STC        83
DLG        77
GST        77
AKN        63
ITH        34
Name: ORIGIN_AIRPORT, Length: 323, dtype: int64

In [36]:
df_taxi.reset_index()
df_taxi_out = (df_taxi.groupby(['ORIGIN_AIRPORT'])['TAXI_OUT_x'].sum()/df_taxi.ORIGIN_AIRPORT.value_counts()).sort_values(ascending = False)

In [37]:
df_taxi_out

LGA    26.733558
JFK    26.728848
ITH    22.514706
PHL    21.554212
ISN    20.950977
         ...    
BET     7.152490
VEL     7.100000
BRW     6.901982
ADQ     6.591398
CNY     6.512195
Length: 323, dtype: float64

In [38]:
df_taxi.query('ORIGIN_AIRPORT == "LGA"').head(1)


,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY_x,TAXI_OUT_x,...,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,DATE
4274714,4,AA,1103,N3HCAA,LGA,DFW,600,NaN,6.406137,25.258086,...,AA,American Airlines Inc.,LGA,LaGuardia Airport (Marine Air Terminal),New York,NY,USA,40.77724,-73.87261,2015-07-05


## Аэропорт с самой большой долей руления на самолетовылет - LaGuardia Airport (Marine Air Terminal) г.Нью-Йорк

In [40]:
my_report = sv.analyze(df_taxi)
my_report.show_html() 

NameError: name 'sv' is not defined

## Будем строить прогноз по показателю ARRIVAL_DELAY_x. После формирования отчета смотрим фичи, где отсутствует корреляция и удаляем из датасета

In [77]:
df_new = df_taxi.drop(['CANCELLATION_REASON', 'CANCELLED','COUNTRY', 'DIVERTED', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ELAPSED_TIME','AIR_TIME', 'SECURITY_DELAY','LATITUDE','LONGITUDE'], axis=1)

In [78]:
df_new.isna().sum()

DAY_OF_WEEK                  0
AIRLINE_x                    0
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME           85576
DEPARTURE_DELAY_x            0
TAXI_OUT_x                   0
WHEELS_OFF               88470
SCHEDULED_TIME               6
DISTANCE                     0
WHEELS_ON                91936
TAXI_IN                  91936
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME             91936
ARRIVAL_DELAY_x              0
AIR_SYSTEM_DELAY       4755046
AIRLINE_DELAY          4755046
LATE_AIRCRAFT_DELAY    4755046
WEATHER_DELAY          4755046
IATA_CODE_x                  0
AIRLINE_y                    0
IATA_CODE_y               4021
AIRPORT                   4021
CITY                      4021
STATE                     4021
DATE                         0
dtype: int64

## Также удалим данные, где очень много пропусков:
# AIR_SYSTEM_DELAY, AIRLINE_DELAY, LATE_AIRCRAFT_DELAY, AIR_SYSTEM_DELAY имеют большой вес влияния на показатель, но 82% пропусков - пока удалим (либо посчитать среднее в зависимости от авиакомпании)
## WEATHER_DELAY - 82% пропусков, но 93,9% значений = 0, поэтому заменим на 0 (т.к. влияние данного значения на показатель ARRIVAL_DELAY_x = 0,26
## 4021 пропуск  в данных по аэропорту BSM, заменим на данные из интернета Austin, TX [Bergstrom Air Rescue Service Airport], USA 

In [79]:
df_taxi[df_taxi.AIRPORT.isna()]

,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY_x,TAXI_OUT_x,...,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,DATE
4182376,4,AS,671,N423AS,BSM,SEA,1840,1837.0,-3.000000,10.000000,...,AS,Alaska Airlines Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-18
4182377,5,AS,671,N402AS,BSM,SEA,1840,1830.0,-10.000000,12.000000,...,AS,Alaska Airlines Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-18
4182378,6,AS,671,N423AS,BSM,SEA,1840,1835.0,-5.000000,8.000000,...,AS,Alaska Airlines Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-18
4182379,7,AS,671,N305AS,BSM,SEA,1840,1835.0,-5.000000,9.000000,...,AS,Alaska Airlines Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-18
4182380,1,AS,671,N483AS,BSM,SEA,1840,1831.0,-9.000000,13.000000,...,AS,Alaska Airlines Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4186392,6,VX,440,N530VA,BSM,DAL,715,NaN,7.380117,11.900585,...,VX,Virgin America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-23
4186393,6,VX,219,N846VA,BSM,SFO,740,NaN,7.380117,11.900585,...,VX,Virgin America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-23
4186394,6,VX,721,N525VA,BSM,DAL,1020,1113.0,53.000000,24.000000,...,VX,Virgin America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-23
4186395,6,VX,449,N523VA,BSM,DAL,1355,1407.0,12.000000,19.000000,...,VX,Virgin America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-23


In [80]:
df_taxi[df_taxi.AIRPORT.isna()].ORIGIN_AIRPORT.unique()

array(['BSM'], dtype=object)

In [81]:
df_new.IATA_CODE_y.fillna('BSM',inplace = True)
df_new.AIRPORT.fillna('Austin, TX [Bergstrom Air Rescue Service Airport]',inplace = True)
df_new.CITY.fillna('Austin',inplace = True)
df_new.STATE.fillna('Texas',inplace = True)

In [82]:
df_new.drop(['AIR_SYSTEM_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY'],axis=1, inplace=True)

In [83]:
df_new['WEATHER_DELAY'].fillna(0,inplace = True)

### хотела заполнить пропуски, но не смогла перевести данные колонки в часы и минуты

In [64]:
#df_new.loc[df_new.DEPARTURE_TIME.isna(),'DEPARTURE_TIME'] = df_new['SCHEDULED_DEPARTURE'] + df_new['DEPARTURE_DELAY_x']

In [84]:
df_new.isna().sum()

DAY_OF_WEEK                0
AIRLINE_x                  0
ORIGIN_AIRPORT             0
DESTINATION_AIRPORT        0
SCHEDULED_DEPARTURE        0
DEPARTURE_TIME         85576
DEPARTURE_DELAY_x          0
TAXI_OUT_x                 0
WHEELS_OFF             88470
SCHEDULED_TIME             6
DISTANCE                   0
WHEELS_ON              91936
TAXI_IN                91936
SCHEDULED_ARRIVAL          0
ARRIVAL_TIME           91936
ARRIVAL_DELAY_x            0
WEATHER_DELAY              0
IATA_CODE_x                0
AIRLINE_y                  0
IATA_CODE_y                0
AIRPORT                    0
CITY                       0
STATE                      0
DATE                       0
dtype: int64

In [53]:
# Удалим нулевые строки из датасета, но проверим, сколько мы потеряем информации после удаления

rows_after_dropping_null = df_new.dropna(axis=0).shape[0]
rows_before_dropping_null = df_new.shape[0]

perc_info_loss = (rows_before_dropping_null - rows_after_dropping_null)*100/rows_before_dropping_null
print("Доля нулевых строк в датасете: {:.2f}%".format(perc_info_loss))

Доля нулевых строк в датасете: 1.59%


## Удалим пропуски из датасета, т.к. это даст незначительную потерю информации 1,59%, но можно восстановить данные, если перевести данные в часы и минуты

In [85]:
df_new.dropna(axis=0,inplace=True)

In [86]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5726549 entries, 0 to 5818484
Data columns (total 24 columns):
 #   Column               Dtype         
---  ------               -----         
 0   DAY_OF_WEEK          int64         
 1   AIRLINE_x            object        
 2   ORIGIN_AIRPORT       object        
 3   DESTINATION_AIRPORT  object        
 4   SCHEDULED_DEPARTURE  int64         
 5   DEPARTURE_TIME       float64       
 6   DEPARTURE_DELAY_x    float64       
 7   TAXI_OUT_x           float64       
 8   WHEELS_OFF           float64       
 9   SCHEDULED_TIME       float64       
 10  DISTANCE             int64         
 11  WHEELS_ON            float64       
 12  TAXI_IN              float64       
 13  SCHEDULED_ARRIVAL    int64         
 14  ARRIVAL_TIME         float64       
 15  ARRIVAL_DELAY_x      float64       
 16  WEATHER_DELAY        float64       
 17  IATA_CODE_x          object        
 18  AIRLINE_y            object        
 19  IATA_CODE_y          

In [87]:
# AIRLINE_x = IATA_CODE_x - удалим 1 колонку
df_new.drop('IATA_CODE_x', axis = 1, inplace = True)

## Преобразуем текстовые данные

In [88]:
df_new1 = pd.get_dummies(df_new, columns=['AIRLINE_x','ORIGIN_AIRPORT','DESTINATION_AIRPORT','AIRLINE_y'])

In [89]:
df_new2 = pd.get_dummies(df_new1, columns=['IATA_CODE_y','AIRPORT','CITY','STATE'])

In [90]:
df_new2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5726549 entries, 0 to 5818484
Columns: 1698 entries, DAY_OF_WEEK to STATE_WY
dtypes: datetime64[ns](1), float64(10), int64(4), uint8(1683)
memory usage: 9.7 GB


## Здесь при проверке на монотонность тоже зависает (я так поняла из-за индекса) и ядро тоже иногда не выдерживает (умирает)

In [51]:

df_new2.set_index('DATE', inplace=True)
df_new2.sort_index(inplace=True)
print('Монотонность - ',df_new2.index.is_monotonic)

Монотонность -  True


In [ ]:
df_new2.reset_index()

In [73]:
# переводим float в int (как выяснилось - это не помогло)
df_new2['DEPARTURE_TIME'] = df_new2['DEPARTURE_TIME'].round(0).astype('int')
df_new2['DEPARTURE_DELAY_x'] = df_new2['DEPARTURE_DELAY_x'].round(0).astype('int')
df_new2['TAXI_OUT_x'] = df_new2['TAXI_OUT_x'].round(0).astype('int')
df_new2['WHEELS_OFF'] = df_new2['WHEELS_OFF'].round(0).astype('int')
df_new2['SCHEDULED_TIME'] = df_new2['SCHEDULED_TIME'].round(0).astype('int')
df_new2['TAXI_IN'] = df_new2['TAXI_IN'].round(0).astype('int')
df_new2['WHEELS_ON'] = df_new2['WHEELS_ON'].round(0).astype('int')
df_new2['ARRIVAL_TIME'] = df_new2['ARRIVAL_TIME'].round(0).astype('int')
df_new2['ARRIVAL_DELAY_x'] = df_new2['ARRIVAL_DELAY_x'].round(0).astype('int')
df_new2['WEATHER_DELAY'] = df_new2['WEATHER_DELAY'].round(0).astype('int')

In [91]:
df_new2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5726549 entries, 0 to 5818484
Columns: 1698 entries, DAY_OF_WEEK to STATE_WY
dtypes: datetime64[ns](1), float64(10), int64(4), uint8(1683)
memory usage: 9.7 GB


In [92]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_new2.drop('ARRIVAL_DELAY_x',axis=1), df_new2['ARRIVAL_DELAY_x'], shuffle=False, test_size=0.25)

In [76]:
clf = RandomForestRegressor()
model = clf.fit(X_train, y_train)
pred = model.predict(X_test)

TypeError: The DTypes <class 'numpy.dtype[datetime64]'> and <class 'numpy.dtype[int64]'> do not have a common DType. For example they cannot be stored in a single array unless the dtype is `object`.

In [ ]:
print('Ошибка RMSE=',mean_squared_error(y_test,pred,squared=False))

### Ну вот выходит вот такая ошибка в predict и на этом я застряла, ниже просто пыталась установить prophet (не получается)

In [1]:
from prophet import Prophet
m = Prophet()

ModuleNotFoundError: No module named 'prophet'

In [2]:
conda install -c conda-forge prophet

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                                     \failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - alabaster -> python[version='2.7.*|3.5.*|3.6.*|>=2.7,<2.8.0a0|>=3.5,<3.6.0a0|>=3.6,<3.7.0a0|3.4.*|>=3.7,<3.8.0a0']
  - anaconda==2021.11 -> python[version='3.7.11|3.8.12|3.9.7',build='h88f2d9e_0|h88f2d9e_1']
  - appdirs -> python[version='2.7.*|3.5.*|3.6.*|3.4.*|>=3.5,<3.6.0a0|>=3.6,<3.7.

appscript -> python[version='>=3.10,<3.11.0a0'] -> xz[version='5.0.*|5.2.*|>=5.2.3,<5.3.0a0|>=5.2.4,<5.3.0a0|>=5.2.5,<5.3.0a0|>=5.2.5,<6.0a0|>=5.2.4,<6.0a0|>=5.2.3,<6.0a0']
argh -> python -> xz[version='5.0.*|5.2.*|>=5.2.3,<5.3.0a0|>=5.2.4,<5.3.0a0|>=5.2.5,<5.3.0a0|>=5.2.5,<6.0a0|>=5.2.4,<6.0a0|>=5.2.3,<6.0a0']
keyring -> pypy3.7[version='>=7.3.7'] -> xz[version='5.0.*|5.2.*|>=5.2.3,<5.3.0a0|>=5.2.4,<5.3.0a0|>=5.2.5,<5.3.0a0|>=5.2.5,<6.0a0|>=5.2.4,<6.0a0|>=5.2.3,<6.0a0']
pysocks -> python[version='>=3.10,<3.11.0a0'] -> xz[version='5.0.*|5.2.*|>=5.2.3,<5.3.0a0|>=5.2.4,<5.3.0a0|>=5.2.5,<5.3.0a0|>=5.2.5,<6.0a0|>=5.2.4,<6.0a0|>=5.2.3,<6.0a0']
pathlib2 -> python[version='>=3.9,<3.10.0a0'] -> xz[version='5.0.*|5.2.*|>=5.2.3,<5.3.0a0|>=5.2.4,<5.3.0a0|>=5.2.5,<5.3.0a0|>=5.2.5,<6.0a0|>=5.2.4,<6.0a0|>=5.2.3,<6.0a0']
pyyaml -> python[version='>=3.7,<3.8.0a0'] -> xz[version='5.0.*|5.2.*|>=5.2.3,<5.3.0a0|>=5.2.4,<5.3.0a0|>=5.2.5,<5.3.0a0|>=5.2.5,<6.0a0|>=5.2.4,<6.0a0|>=5.2.3,<6.0a0']
xmltodict -> p

pyzmq -> python[version='>=3.7,<3.8.0a0'] -> readline[version='6.2.*|7.0|>=7.0,<8.0a0|>=8.0,<9.0a0|>=8.1,<9.0a0|7.*|7.0.*']
typed-ast -> python[version='>=3.10,<3.11.0a0'] -> readline[version='6.2.*|7.0|>=7.0,<8.0a0|>=8.0,<9.0a0|>=8.1,<9.0a0|7.*']
prompt-toolkit -> python[version='>=3.6'] -> readline[version='6.2.*|7.0|>=7.0,<8.0a0|>=8.0,<9.0a0|>=8.1,<9.0a0|7.*|7.0.*']
importlib_metadata -> python[version='>=3.8,<3.9.0a0'] -> readline[version='6.2.*|7.0|7.0.*|>=7.0,<8.0a0|>=8.0,<9.0a0|>=8.1,<9.0a0|7.*']
watchdog -> python[version='>=3.10,<3.11.0a0'] -> readline[version='6.2.*|7.0|>=7.0,<8.0a0|>=8.0,<9.0a0|>=8.1,<9.0a0|7.*|7.0.*']
json5 -> python -> readline[version='6.2.*|7.0|>=7.0,<8.0a0|>=8.0,<9.0a0|>=8.1,<9.0a0|7.0.*|7.*']
tinycss -> python[version='>=3.6'] -> readline[version='6.2.*|7.0|>=7.0,<8.0a0|>=8.0,<9.0a0|>=8.1,<9.0a0|7.*|7.0.*']
matplotlib-inline -> python[version='>=3.6'] -> readline[version='6.2.*|7.0|>=7.0,<8.0a0|>=8.0,<9.0a0|>=8.1,<9.0a0|7.*']
qdarkstyle -> python[versi

pluggy -> python[version='>=3.10,<3.11.0a0'] -> tzdata
ujson -> python[version='>=3.9,<3.10.0a0'] -> tzdata
wrapt -> python[version='>=3.10,<3.11.0a0'] -> tzdata
dask -> python[version='>=3.7'] -> tzdata
jinja2 -> python[version='>=3.6'] -> tzdata
typing_extensions -> python[version='>=3.6'] -> tzdata
requests -> python[version='>=3.6'] -> tzdata
more-itertools -> python[version='>=3.4'] -> tzdata
gevent -> python[version='>=3.10,<3.11.0a0'] -> tzdata
babel -> python -> tzdata
regex -> python[version='>=3.9,<3.10.0a0'] -> tzdata
scikit-learn -> python[version='>=3.10,<3.11.0a0'] -> tzdata
ipython_genutils -> python -> tzdata
jupyter_server -> python[version='>=3.7'] -> tzdata
pytest -> python[version='>=3.10,<3.11.0a0'] -> tzdata
sympy -> python[version='>=3.10,<3.11.0a0'] -> tzdata
zope.event -> python -> tzdata
py -> python[version='>=2.7'] -> tzdata
mkl-service -> python[version='>=3.10,<3.11.0a0'] -> tzdata
argh -> python -> tzdata
imagesize -> python[version='>=3.4'] -> tzdata
pat

matplotlib-base -> numpy[version='>=1.17'] -> numpy-base[version='1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.11.3|1.14.3|1.14.3|1.14.3|1.14.3|1.14.3|1.14.3|1.14.4|1.14.4|1.14.4|1.14.4|1.14.4|1.14.4|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.5|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.14.6|1.15.0|1.15.0|1.15.0|1.15.0|1.15.0|1.15.0|1.15.0|1.15.0|1.15.1|1.15.1|1.15.1|1.15.1|1.15.1|1.15.1|1.15.1|1.15.1|1.15.1|1.15.1|1.15.1|1.1


Note: you may need to restart the kernel to use updated packages.
